# Part 3

Explore and Cluster

In [31]:
!pip install folium
from bs4 import BeautifulSoup
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Libraries imported.


In [32]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')
column_list = ['Postalcode', 'Borough', 'Neighborhood']
df = pd.DataFrame(columns = column_list)
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postalcode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [33]:
geo_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

In [34]:
geo_df.rename(columns = {'Postal Code' : 'Postalcode'}, inplace = True)
merged = pd.merge(geo_df, df, on = 'Postalcode')

In [35]:
final_data = merged[['Postalcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]

# Use geopy library to get the latitude and longitude values of Toronto.

In [36]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


# Getting all rows contain Toronto

In [37]:
part3_df = final_data[final_data['Borough'].str.contains('Toronto', regex = False)]


# Creating a map of Toronto with neighbourhoods

In [38]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.6534817, -79.3839347], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(part3_df['Latitude'], part3_df['Longitude'], part3_df['Borough'], part3_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Using KMeans to cluster

In [39]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')

Libraries imported.


In [40]:
num_clusters = 10
toronto = part3_df.drop(['Postalcode', 'Borough', 'Neighborhood'], 1)
k_means = KMeans(n_clusters = num_clusters, n_init = 12)
k_means.fit(toronto)
labels = k_means.labels_
part3_df.insert(0, 'Cluster', labels)

In [41]:
part3_df

,Cluster,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,5,M4E,East Toronto,The Beaches,43.676357,-79.293031
40,2,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106
41,2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,5,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,2,M4M,East Toronto,Studio District,43.659526,-79.340923
44,6,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,6,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,6,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,7,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


In [45]:
# create map of Toronto
map_toronto = folium.Map(location=[43.6534817, -79.3839347], zoom_start=10)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lon, poi, cluster in zip(part3_df['Latitude'], part3_df['Longitude'], part3_df['Neighborhood'], part3_df['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)
map_toronto